In [1]:
import numpy as np
import pysam, os, pickle, sys
from Bio.Seq import Seq
from Bio import SeqIO
from fuzzysearch import find_near_matches
from dataclasses import dataclass
from copy import deepcopy

@dataclass
class read_part:
    cigar: str
    read_start: int
    read_end: int
    chrom: str=''
    strand: chr=''
    ref_start: int = -1
    ref_end: int = -1
    mapq: int = -1
    
    def get_length(self):
        return self.read_end-self.read_start
    
sys.path.append('scripts')
from process_reads import *

In [2]:
transgene_ref = 'references/CBh_template+rDNAflanks.fa'
flank_len = 840
transgene_len = len(list(SeqIO.parse(transgene_ref, format='fasta'))[0].seq) - 2*flank_len
tg_end = flank_len+transgene_len
transgene_refname = list(SeqIO.parse(transgene_ref, format='fasta'))[0].name

rDNA_repeat = list(SeqIO.parse('references/rDNA_fullrepeat.fa', format='fasta'))[0].seq
rDNA_repeat = str(rDNA_repeat).upper()

template_flanks = SeqIO.parse(transgene_ref, format='fasta')
template_flanks = str(list(template_flanks)[0].seq).upper()

In [3]:
# read sam file and convert to dataframe representation
input_sam = 'KCXZ0001D_transgeneflanks_mappedmates.sam'
transgenereads = pysam.AlignmentFile(input_sam, 'r')
read_dict = populate_read_dict(transgenereads)

In [4]:
read_dict = realign_reads(read_dict, 'references/rDNA_fullrepeat.fa')
read_dict = realign_reads(read_dict, transgene_ref)
read_dict = realign_reads(read_dict, 'references/chm13v2.0.fa.gz')

# read_dict = discard_plasmid_reads(read_dict, '../../../templates/CBh_plasmid.fa')
read_dict = discard_contaminating_reads(read_dict, 'references/blacklist.fa')

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 79858 sequences (10000027 bp)...
[M::process] read 74998 sequences (10000237 bp)...
[M::mem_process_seqs] Processed 79858 reads in 5.667 CPU sec, 5.654 real sec
[M::process] read 42638 sequences (5956691 bp)...
[M::mem_process_seqs] Processed 74998 reads in 3.287 CPU sec, 3.257 real sec
[M::mem_process_seqs] Processed 42638 reads in 0.864 CPU sec, 0.843 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -B 6 references/rDNA_fullrepeat.fa temp.fa
[main] Real time: 9.785 sec; CPU: 9.848 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 77798 sequences (10000044 bp)...
[M::process] read 65667 sequences (9194667 bp)...
[M::mem_process_seqs] Processed 77798 reads in 1.568 CPU sec, 1.555 real sec
[M::mem_process_seqs] Processed 65667 reads in 1.551 CPU sec, 1.534 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -B 6 references/CBh_template+rDNAflanks.fa temp.fa
[main] Real time: 3.125 sec; CPU

In [5]:
read_dict = align_clips(read_dict,  'references/rDNA_fullrepeat.fa', transgene_refname)
read_dict = align_clips(read_dict,  'references/chm13v2.0.fa.gz', transgene_refname)
read_dict = align_clips(read_dict, transgene_ref, transgene_refname)

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 34067 sequences (1331281 bp)...
[M::mem_process_seqs] Processed 34067 reads in 0.200 CPU sec, 0.200 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -B 6 references/rDNA_fullrepeat.fa temp.fa
[main] Real time: 0.212 sec; CPU: 0.213 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 33689 sequences (1309623 bp)...
[M::mem_process_seqs] Processed 33689 reads in 0.659 CPU sec, 0.661 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -B 6 references/chm13v2.0.fa.gz temp.fa
[main] Real time: 1.777 sec; CPU: 1.580 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 33509 sequences (1300512 bp)...
[M::mem_process_seqs] Processed 33509 reads in 0.156 CPU sec, 0.156 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -B 6 references/CBh_template+rDNAflanks.fa temp.fa
[main] Real time: 0.169 sec; CPU: 0.169 sec


In [6]:
read_dict = filter_transgenemapping(read_dict, flank_len, tg_end, transgene_refname)
read_dict = realign_rDNA_reads(read_dict, transgene_refname, flank_len, tg_end, rDNA_repeat)
read_dict = remap_sensitive(read_dict, template_flanks, rDNA_repeat, transgene_refname)
read_dict = filter_transgenemapping(read_dict, flank_len, tg_end, transgene_refname)

with open('KCXZ0001D_reads.pkl', 'wb') as f:
    pickle.dump(read_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

print(len(read_dict))

4883
